# For training BioBERT NER on nala
Run this in CLI -  

python train_ner.py --model_name_or_path dmis-lab/biobert-base-cased-v1.1 --train_file data/nala/train_dev.json --validation_file data/nala/devel.json --text_column_name tokens --label_column_name tags --pad_to_max_length --max_length 192 --per_device_train_batch_size 8 --learning_rate 2e-5 --num_train_epochs 10 --output_dir models/nala --seed 1
  
Note:  Training took <30 mins on 1660 ti. You can decrease the num_train_epochs count to 5 without any substantial difference in accuracy.

# 1 Regex 
## Has high precision and low recall.    
## Consists of 3 parts - MutationFinder, tmVar and some custom patterns from WB papers 
### 1.1 Mutation Finder [Link](https://github.com/divyashan/MutationFinder), Modified regex from SETH [Link](https://github.com/rockt/SETH/blob/master/resources/mutations.txt)

In [1]:
import re
import os
import time
 
import numpy as np
import pandas as pd
import glob
from pathlib import Path

from sklearn.metrics import precision_recall_fscore_support

In [2]:
from utils.misc.regex_block import mutation_finder_from_regex_filepath, TmVar, BOWdictionary, CustomWBregex

In [3]:
mf_regex_path = 'data/regexs/mutationfinder_regex/seth_modified.txt'
mf_mut_extract = mutation_finder_from_regex_filepath(mf_regex_path)

In [4]:
text = 'A(1154)C'
for mutation, snip in mf_mut_extract(raw_text=text, span_size=150).items():
    mutation_entry = snip + ' : ' + mutation.OriginalMention
    print(mutation_entry)

A(1154)C : A(1154)C


### 1.2 tmVar [Link](https://www.ncbi.nlm.nih.gov/research/bionlp/Tools/tmvar/)

In [5]:
tmvar_mut_extract = TmVar('data/regexs/tmvar_regex/final_regex_path')

/mnt/c/Users/Rishab/Documents/GitHub/genomic-info-from-papers/genomic-info-from-papers/utils/misc/regex_block.py:302: FutureWarning: Possible nested set at position 15
  self._regular_expressions.append(re.compile(reg))


In [6]:
tmvar_mut_extract(' n2923 (A347V), n2870(R429K), and n1163(S486F')

[['A347V', ' n2923 (A347V), n2870(R429K), and n1163(S486F'],
 ['R429K', ' n2923 (A347V), n2870(R429K), and n1163(S486F']]

### 1.3 Extra custom regexs 

In [7]:
import configparser

In [8]:
db_config = configparser.ConfigParser()
db_config.read('utils/all_config.cfg')

['utils/all_config.cfg']

In [9]:
custom_mut_extract = CustomWBregex(db_config, extra_regex=True)

- in the statement ced-3(n2888) 
n2888 bp AAAAAAAAAAAAAAAAA TTTTTTTTTTTTTTTTTT

In [10]:
custom_mut_extract('n2888 bp AAAAAAAAAAAAAAAAA TTTTTTTTTTTTTTTTTT')

[['n2888 bp AAAAAAAAAAAAAAAAA TTTTTTTTTTTTTTTTTT',
  'n2888 bp AAAAAAAAAAAAAAAAA TTTTTTTTTTTTTTTTTT']]

In [11]:
custom_mut_extract.get_genes('ced-3')

[['ced-3', 'Just gene']]

### 1.4 Bag of words 

In [12]:
bow_mut_extract = BOWdictionary()
# bow_mut_extract('This mutation deletes 471bp of the promoter region, the transcriptional start and 56 amino acids of the second exon.')

### 1.6 MF + tmVar + Custom regex + BOW

In [13]:
def unique_rows(a):
    a = np.ascontiguousarray(a)
    unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
    return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))


def regex_block(sentence, span_size=150):
    mut_and_snippets = []
    
    # MutationFinder
    for mutation, snip in mf_mut_extract(raw_text=sentence, span_size=span_size).items():
        mut_and_snippets.append([mutation.OriginalMention, snip])
    
    # tmVar
    mut_and_snippets = mut_and_snippets + tmvar_mut_extract(sentence, span_size=span_size)
    # Custom patterns
    mut_and_snippets = mut_and_snippets + custom_mut_extract(sentence, span_size=span_size)
    # Bag of words
    mut_and_snippets = mut_and_snippets + bow_mut_extract(sentence)
    
    if mut_and_snippets:
        mut_and_snippets = unique_rows(mut_and_snippets).tolist()
    return mut_and_snippets

In [14]:
regex_block(' asdf gpa-2 ::Tc1 asdf as')

[]

### 1.7 * Additional details  

In [15]:
def extra_info_block(sentence, span_size=150):
    info_and_snippets = []

    # look for gene and variant combo
    info_and_snippets = info_and_snippets + custom_mut_extract.var_and_gene_close(sentence, span_size=span_size)
    
    if info_and_snippets:
        info_and_snippets = unique_rows(info_and_snippets).tolist()
    return info_and_snippets

In [16]:
extra_info_block('in the statement ced-3(n2888)')

[['ced-3(n2888', 'Gene & Variant']]

## Changes in scores
MF + tmVar only: on remarks - (0.9459459459459459, 0.39923954372623577, 0.5614973262032086, None)   
on remarks + nala - (0.8671477079796265, 0.6698360655737705, 0.755826859045505, None)  
    
MF + tmVar + Custom + BOW: on remarks - (0.9606741573033708, 0.6501901140684411, 0.7755102040816326, None)  
on remarks + nala - (0.8680868496517821, 0.6947540983606557, 0.7718084137679839, None)  

# 2 BioBERT NER

In [17]:
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification
from transformers import TokenClassificationPipeline

import re
import pandas as pd
import numpy as np
import sklearn as sk
import math 
import string
import time
import json
import csv
import shutil
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/risubu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
model_name_or_path = 'models/nala'
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForTokenClassification.from_pretrained(
    model_name_or_path,
    from_tf=bool(".ckpt" in model_name_or_path),
    config=config,
)
# LABEL_0 - B-mut, LABEL_1 - I-mut, LABEL_2 - O
nala_ner  = TokenClassificationPipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='first')

In [19]:
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words = [w for w in stop_words if len(w) > 1]

def ner_score(sentence):
    mutations = []
    try:
        ner_output = nala_ner(sentence)
        for i, grp in enumerate(ner_output):
            if grp['entity_group'] == 'LABEL_0':
                mut = grp['word']
                for j in range(i+1, len(ner_output)):
                    if ner_output[j]['entity_group'] == 'LABEL_1':
                        mut  = mut + ' ' + ner_output[j]['word']
                    else:
                        # NER would be handling only data in NL form
                        if len(mut.split()) > 3 and any(word in mut.split() for word in stop_words):
                            mutations.append([mut, sentence])
                        break
    except:
        pass
    return mutations

# 3 Testing 
## 3.2 On scored curator remarks and nala mutation corpus 

In [20]:
# all_texts = []
# issues_count = 0

# df = pd.read_csv(r"data\gsoc\Remarks_scored.csv")
# df = df.to_numpy()
# # extract sentences
# text = []
# y_true = []
# for idx, row in enumerate(df):
#     loc = str(row[2]).find('Paper_evidence')
#     if loc != -1:
#         if row[0].split()[0] == 'Yes':
#             y_true.append(1)
#         elif row[0].split()[0] == 'No':
#             y_true.append(0)
#         else:
#             continue
#         temp_str = str(row[2][1:loc-2]).replace("\"", "'")
#         text.append(temp_str)

# assert len(text) == len(y_true)
# print('Count from Remarks_scored = {}'.format(len(text)))

# y_pred = []
# for sentence in text:
#     if regex_block(sentence):
#         y_pred.append(1)
#     elif ner_score(sentence):
#         y_pred.append(1)
#     else:
#         y_pred.append(0)

# assert len(y_pred) == len(y_true)
# all_texts = all_texts + text


# df = pd.read_csv(r"data\nala\binary_nala_NOT_NER.csv")
# df = df.to_numpy()
# print('Count from nala = {}'.format(len(df)))

# print('Entries processed: ', end=' ')
# for i, row in enumerate(df):
#     if i%500 == 0: print(i, end=' ')
#     sentence = row[0]
#     label = row[1]
#     try:
#         if regex_block(sentence):
#             y_pred.append(1)
#         elif ner_score(sentence):
#             y_pred.append(1)
#         else:
#             y_pred.append(0)

#         if label == 1:
#             y_true.append(1)
#         else:
#             y_true.append(0)
        
#         all_texts.append(sentence)
#     except:
#         issues_count += 1
#         pass

# assert len(y_pred) == len(y_true) == len(all_texts)
# print('\nTotal count = {}'.format(len(y_pred)))
# if issues_count: print('Note: Could not process {} sentences'.format(issues_count))

# precision_recall_fscore_support(y_true, y_pred, average='binary')

In [21]:
# # Manually inspecting incorrect preds
# bad = []
# assert len(y_pred) == len(y_true) == len(all_texts)
# for i, (t, p,sent) in enumerate(zip(y_true, y_pred, all_texts)):
#     if t != p:
#         if t:
#             bad.append([sent,i])
# len(bad)
# # print(bad[:10])

## Changes in scores (find the test cell block below)
  
Regex only: on remarks - (0.9606741573033708, 0.6501901140684411, 0.7755102040816326, None)  
on remarks + nala - (0.8680868496517821, 0.6947540983606557, 0.7718084137679839, None) 
  
Regex + NER: on remarks - (0.8592057761732852, 0.9049429657794676, 0.8814814814814815, None)   
on remarks + nala - (0.7570694087403599, 0.9655737704918033, 0.8487031700288185, None)  
  
### Retraining on NL and SST  
Regex + NER: on remarks - (0.9627906976744186, 0.7870722433460076, 0.8661087866108786, None)      
on remarks + nala - (0.8461816865725661, 0.7665573770491804, 0.804403922243248, None)  

## 3.2 On WB papers
### Get the paper texts from textpresso API and wbtools

### How it works - First paper ID is searched through textpresso API. If the recieved output is blank, then wbtools is used.  

In [22]:
import sys
import os.path
import argparse
import subprocess
import requests
import csv
import json
import nltk.data
import os
from xml.dom import minidom
from bs4 import BeautifulSoup
from wbtools.literature.corpus import CorpusManager
import platform

Uncomment and run the cell below only once to save the paper sentences in your local computer to avoid pulling paper text everytime during dev  

In [23]:
'''
Create a separate temporary numpy file to store the paper sentences  
to avoid spending time on pulling sentences while in development 
Format - [paper_id, sentence]
'''
# final_data = []
# # remove_sections = [PaperSections.ACKNOWLEDGEMENTS, PaperSections.REFERENCES, PaperSections.RELATED_WORK, PaperSections.INTRODUCTION]
# remove_sections = []
# # random 100 papers mentioned the remarks ace file in data/gsoc
# paper_ids = np.load('data/top100.npy').tolist()
# cm = CorpusManager()
# for i, paper_id in enumerate(paper_ids):
#     paper_id = paper_id[7:]
#     cm.load_from_wb_database(db_name=db_config['wb_database']['db_name'], db_user=db_config['wb_database']['db_user'], db_password=db_config['wb_database']['db_password'],
#         db_host=db_config['wb_database']['db_host'], paper_ids=[paper_id],
#         ssh_host=db_config['wb_database']['ssh_host'], ssh_user=db_config['wb_database']['ssh_user'], ssh_passwd=db_config['wb_database']['ssh_passwd'],
#         load_bib_info=False, load_afp_info=False, load_curation_info=False)
#     sentences = cm.get_paper(paper_id).get_text_docs(remove_sections=remove_sections,split_sentences=True)
#     for sent in sentences:
#         final_data.append([paper_id, sent])
#     print(i, end = " ")
# final_data = pd.DataFrame(final_data[:], columns=['WBPaper ID', 'Sentence'])
# final_data.to_csv("data/id_and_sentence.csv", index=False, encoding='utf-8')

'\nCreate a separate temporary numpy file to store the paper sentences  \nto avoid spending time on pulling sentences while in development \nFormat - [paper_id, sentence]\n'

In [24]:
def textpresso_paper_text(wbpid, path, token):
    """This sub takes a wbpid eg WBPaper00056731 and returns the fulltext paper in sentences"""
    ft=[0];
    # Check that wbpid is a valid WBPaper
    if not re.match( 'WBPaper', wbpid):
        print (wbpid, "is not a valid WBPaper ID")
        return ft
    # Download paper if it doesn't exist
    fn = path + '/temp/' + wbpid + '.json'

    if os.path.exists(fn) and os.path.getsize(fn) > 16:
        pass
    else:
        com1 = '-o '+fn +'\n-k '+ '\n'+'-d "{\\"token\\":\\"'+ token + '\\", \\"query\\": {\\"accession\\": \\"' + wbpid +'\\", \\"type\\": \\"document\\", \\"corpora\\": [\\"C. elegans\\"]}, \\"include_fulltext\\": true}"'
        configf= path +'/temp/' + wbpid + '.tmp.config'
        curlf = open(configf,'w')
        print (com1, file=curlf)
        curlf.close()
        command = 'curl -o '+ fn +' -K '+ configf+' https://textpressocentral.org:18080/v1/textpresso/api/search_documents' 
        comlist = command.split()
        os.system(command)

    # Read the paper, and split into sentences
    if os.path.exists(fn) and os.path.getsize(fn) > 20:
        # Open our JSON file and load it into python
        input_file = open (fn)
        json_array = json.load(input_file)
        for item in json_array:
            abs = item["abstract"]
            fullt =  item["fulltext"]
            tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
            ft = tokenizer.tokenize(abs)
            ftt=tokenizer.tokenize(fullt)
            ft = ft +ftt
    else:
        # some paper texts are blank for some reason
        # pipeline uses wbtools to get sentences in such case
        pass

    outfilen = os.path.join(path, 'text_flatfiles', wbpid+'.txt')
    outf = open(outfilen, 'w')
    for sen in ft:
        sen =str(sen)
        print(sen, file=outf)
    outf.close()
    return outfilen

In [25]:
'''
Sentences out of wbtools are sometimes weird, especially for the old papers
This increases the false neg rate so this is used only when the textpresso api provides bad text
Most of the issues have band aid fixes in the text preprocessing cell below but there are some 
with no easy fix or need to be worked on - 
1. Table content is extracted column wise, not row wise 
2. Some sentences have white space between every character and are also somehow inverted (???)
    e.g. Check out the sentences of WBPaper00002018 (1994)
    Line 154 -  '0 7 1 u ( 7 - c e m    ) F (   .'
    inverted and without the white space is (mec-7)u170 which is extremely useful and will get missed 
    by the pipeline unless processed correctly.
TODO: Not sure how to solve point 1 but point 2 is easy to solve and also helps a LOT.
    Rishab, work on this after you complete the project. 
    Not high priority as this might be only for the >10 year old papers (which are already manually curated)
'''
cm = CorpusManager()
def wbtools_paper_text(wbpid, db_name, db_user, db_password, db_host, ssh_host,\
    ssh_user, ssh_passwd):
    # sectioning might not be always correct, text processing is done separately in the pipeline
    # remove_sections = [PaperSections.ACKNOWLEDGEMENTS, PaperSections.REFERENCES, PaperSections.RELATED_WORK, PaperSections.INTRODUCTION]
    remove_sections = []
    paper_id = wbpid[7:]
    cm.load_from_wb_database(db_name=db_name, db_user=db_user, db_password=db_password,
        db_host=db_host, paper_ids=[paper_id],
        ssh_host=ssh_host, ssh_user=ssh_user, ssh_passwd=ssh_passwd,
        load_bib_info=False, load_afp_info=False, load_curation_info=False)
    sentences = cm.get_paper(paper_id).get_text_docs(remove_sections=remove_sections,split_sentences=True)
    return sentences

In [26]:
def get_paper_sentences(wbpids, config, store_ppr_path):
    '''
    Takes WB Paper IDs and returns a list of sentences from those papers after filtering
    Arg:
    wbpids - List of wb papers ids 
        e.g. ['WBPaper00002379']
    config_path - Config file path
    store_ppr_path - Folder path to store the paper flatfiles retrieved from TextPresso for future use
    Returns:
    paperid_sentence_list: List of paper ID and sentence
        e.g. [['WBPaper00002379', 'First sentence'], ['WBPaper00002379', 'Second sentence'], ....]
    '''
    stop_words = set(nltk.corpus.stopwords.words('english'))
    stop_words = [w for w in stop_words if len(w) > 1]

    all_special_chars = []
    with open('data/nala/train_dev.json') as f:
        for jsonObj in f:
            nala_json = json.loads(jsonObj)['tokens']
            for word in nala_json:
                if not word.isalnum():
                    all_special_chars.append(word)
    # list of special characters to keep during inference
    # helps with clearing out the bad characters from old papers
    all_special_chars = list(set(all_special_chars))
    
    token = config['textpresso']['token']
    db_name=config['wb_database']['db_name']
    db_user=config['wb_database']['db_user']
    db_password=config['wb_database']['db_password']
    db_host=config['wb_database']['db_host']
    ssh_host=config['wb_database']['ssh_host']
    ssh_user=config['wb_database']['ssh_user']
    ssh_passwd=config['wb_database']['ssh_passwd']

    temp_paperid_sentence = np.array([])
    if os.path.isfile('data/id_and_sentence.csv'):
        temp_paperid_sentence = pd.read_csv("data/id_and_sentence.csv", lineterminator='\n', dtype = str).to_numpy() # WBPaper ID, Sentence
    paperid_sentence_list = []
    
    for curr_ppr_i, id in enumerate(wbpids):
        print(f"{curr_ppr_i+1}", end = " ")
        # textpresso_paper_text() also saves the text in flatfiles for future use 
        start_time = time.time()
        paper_path = textpresso_paper_text(id, store_ppr_path, token)
        txt = Path(paper_path).read_text().split('\n')
        # deals with empty text files with only "0"
        if len(txt) == 2:
            if temp_paperid_sentence.size != 0:
                txt = temp_paperid_sentence[temp_paperid_sentence[:, 0] == id[7:]][:, 1]
                # incase the loaded numpy file didn't have the required paper 
                if len(txt) == 0 and platform.system() != 'Windows':
                    txt = wbtools_paper_text(id, db_name, db_user, db_password, db_host, ssh_host,\
                        ssh_user, ssh_passwd)
            elif platform.system() != 'Windows' :
                txt = wbtools_paper_text(id, db_name, db_user, db_password, db_host, ssh_host,\
                    ssh_user, ssh_passwd)
            
        for row in txt: 
            if row.find('fifi') != -1:
                if temp_paperid_sentence.size != 0:
                    txt = temp_paperid_sentence[temp_paperid_sentence[:, 0] == id[7:]][:, 1]
                    # incase the loaded numpy file didn't have the required paper 
                    if len(txt) == 0 and platform.system() != 'Windows':
                        txt = wbtools_paper_text(id, db_name, db_user, db_password, db_host, ssh_host,\
                            ssh_user, ssh_passwd)
                elif platform.system() != 'Windows':
                    txt = wbtools_paper_text(id, db_name, db_user, db_password, db_host, ssh_host,\
                        ssh_user, ssh_passwd)
                break
            
        count_total_rows = len(txt)
        for current_i, row in enumerate(txt):
            if row.lower().find("we thank") == 0 or row.lower().find("this work was supported") == 0 \
                or row.lower().find("references") == 0 or row.lower().find("we also thank") == 0 \
                or row.lower().find("this research was supported") == 0 or row.lower().find("we acknowledge") == 0 \
                or row.lower().find("acknowledgments") == 0 or row.lower().find('literature cited') != -1:
                if current_i > count_total_rows/3:
                    break

            # usually is bad sentence
            if len(row) < 40 or not any(word in row.lower().split() for word in stop_words):
                continue
            # remove sentences with links and email ids
            if re.search('\S+@\S+\.', row) or re.search('www\.\S+\.', row):
                continue
            # filters one word sentences
            if len(row.split()) == 1:
                continue
            # sentences comprised of only single characters 
            # ^ seems to be issue with wbtools extraction pipeline 
            if all(len(word) < 5 for word in row.split()):
                continue
            row = re.sub("\( *cid *: *\d+ *\)", " ", row)
            # TODO: replace this block with a regex sub 
            temp_row = row
            for c in temp_row:
                if (not c.isalnum() and not c == ' ') and c not in all_special_chars:
                        row = row.replace(c, "")
                        
#             fixes bad space between each character of flanking sequence from old papers
#             Switching this off as it increases the processing time
#             also affects very small subset of old papers so not worth the extra time
            flanking_regex = re.compile('([ACTG]( +)){4,}')
            for m in flanking_regex.finditer(row):
                span = (m.start(0), m.end(0))   
                span = row[span[0]:span[1]-1]
                correct_flank = re.sub('([ACTG])( +)', r'\1', row)
                row = row.replace(span, correct_flank)
            paperid_sentence_list.append((id, row))
    return paperid_sentence_list[1:]

In [27]:
# papers mentioned the remarks ace file in data/gsoc
ids_to_extract = np.load('data/top100.npy').tolist()[:-20]
paperid_sentence_list = get_paper_sentences(ids_to_extract, db_config, store_ppr_path='data/wbpapers')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100     4  100   148      1     72  0:00:04  0:00:02  0:00:02    74


16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 

In [28]:
# remove duplicates keeping the order
seen = set()
paperid_sentence_list = np.array([x for x in paperid_sentence_list if x not in seen and not seen.add(x)])

In [29]:
print('Number of sentences and characters: ', end=' ')
print(len(paperid_sentence_list), sum([len(sent[1]) for sent in paperid_sentence_list]))

Number of sentences and characters:  25476 4097379


This cell takes a while to run - mainly due to the huge regex blocks   
~1 hour per 10 papers   
Can't really switch them off though. There might be a smarter way to work with the regex block but this pipeline would be probably running every month on 50 or so papers so low priority.   

In [30]:
start_time = time.time()
final = [
    ['temporary', 'temporary', 'temporary', 'temporary', 'temporary', 'temporary'],\
    ['WBPaper ID', 'Method', '*Genes', '*Gene-Variant combo', 'Mutation', 'Sentence']]
total_sentences = len(paperid_sentence_list)
print_snips = False
print('Total sentences to process ', len(paperid_sentence_list))
# there would be lots of duplicates in the gene col which might cause high count of
# snippets with useful info. they'll get filtered out in the third notebook
print('{total sentences processed}>{snippets with useful info}')
for ppr_sen_idx, row in enumerate(paperid_sentence_list):
    if (ppr_sen_idx+1) % 50 == 0: print(f"{ppr_sen_idx+1}>{len(final)-1}", end = " ")
    paper_id = row[0]
    sentence = str()
    limit = min(ppr_sen_idx+2, total_sentences)
    # some sentences - mostly table content are super long
    # temp fix, need to have a nice sentence splitter to minimize manual verification time
    not_single_sentence = False
    for i in range(ppr_sen_idx, limit):

        sentence = sentence + paperid_sentence_list[i][1] + ' '

        if (len(sentence) > 250 and not_single_sentence):
            break
        if paper_id != paperid_sentence_list[i][0]:
            break
        
        var_plus_genes = ''
        # Look for the special data e.g. gene-variant combo (e.g 'ced-3(n2888)') only on single sentences 
        if not not_single_sentence:
            var_plus_genes = []
            all_genes = []
            
            for data_and_cat in custom_mut_extract.var_and_gene_close(sentence.strip()):
                var_plus_genes.append(data_and_cat[0])
            if var_plus_genes:
                var_plus_genes  = list(set(var_plus_genes))
                var_plus_genes = "'" + "', '".join(var_plus_genes) + "'"
            else:
                var_plus_genes = ''
                
            for data_and_cat in custom_mut_extract.get_genes(sentence.strip()):
                all_genes.append(data_and_cat[0])
            if all_genes:
                all_genes  = list(set(all_genes))
                all_genes = "'" + "', '".join(all_genes) + "'"
            else:
                all_genes = ''   
                
        output = regex_block(sentence.strip())
        if output:
            mutations = []
            for mut_and_snip in output:
                # temp fix to deal with same mutation getting detected due to stiching multiple sentences
                if (mut_and_snip[0] not in final[-1][4][1:-1].split(", ") and mut_and_snip[0] not in final[-2][4][1:-1].split(", ")) \
                            and mut_and_snip[0] not in mutations:
                    mutations.append(mut_and_snip[0])
            if mutations:
                mutations = "'" + "', '".join(mutations) + "'"
                if print_snips: print(1, mutations)
                final.append([paper_id, 'Regex', all_genes, var_plus_genes, mutations, 'Line '+str(ppr_sen_idx)+': '+sentence.strip()])
            break

        output = ner_score(sentence.strip())
        if output:
            mutations = []
            for mut_and_snip in output:
                # temp fix to deal with same mutation getting detected due to stiching multiple sentences
                if (mut_and_snip[0] not in final[-1][4][1:-1].split(", ") and mut_and_snip[0] not in final[-2][4][1:-1].split(", ")) \
                        and not all(len(word) < 4 for word in mut_and_snip[0].split())\
                    and mut_and_snip[0] not in mutations:
                    mutations.append(mut_and_snip[0])
            if mutations:
                mutations = "'" + "', '".join(mutations) + "'"
                if print_snips: print(2, mutations)
                final.append([paper_id, 'NER', all_genes, var_plus_genes, mutations, 'Line '+str(ppr_sen_idx)+': '+sentence.strip()])
            break
        
        # these data, if found, are going to be important if no mutations are in that sentence
        if var_plus_genes or all_genes:
            final.append([paper_id, '', all_genes, var_plus_genes, '', 'Line '+str(ppr_sen_idx)+': '+sentence.strip()])
        
        not_single_sentence = True
print('Total time for processing in minutes: ', int((time.time() - start_time)/60))

Total sentences to process  25476
{total sentences processed}>{snippets with useful info}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


50>8 100>13 150>17 200>21 250>28 300>40 350>55 400>56 450>69 500>94 550>101 600>112 650>146 700>160 750>196 800>244 850>290 900>335 950>367 1000>371 1050>380 1100>386 1150>395 1200>414 1250>416 1300>416 1350>433 1400>454 1450>459 1500>471 1550>477 1600>484 1650>486 1700>497 1750>505 1800>523 1850>524 1900>544 1950>570 2000>599 2050>643 2100>685 2150>737 2200>779 2250>824 2300>842 2350>854 2400>880 2450>905 2500>945 2550>979 2600>993 2650>1017 2700>1037 2750>1069 2800>1099 2850>1148 2900>1192 2950>1228 3000>1250 3050>1266 3100>1295 3150>1296 3200>1305 3250>1325 3300>1329 3350>1370 3400>1412 3450>1449 3500>1459 3550>1478 3600>1499 3650>1502 3700>1502 3750>1504 3800>1505 3850>1506 3900>1509 3950>1523 4000>1534 4050>1550 4100>1570 4150>1594 4200>1614 4250>1634 4300>1653 4350>1684 4400>1724 4450>1767 4500>1801 4550>1834 4600>1860 4650>1870 4700>1919 4750>1970 4800>2016 4850>2043 4900>2066 4950>2088 5000>2109 5050>2155 5100>2208 5150>2253 5200>2292 5250>2306 5300>2318 5350>2351 5400>2382 545

In [31]:
temp = final[2:] # removing the temporary first row and header

# this sheet will contain high number of duplicates - which will get filtered in 3rd notebook
# columns with asterisk contain data which are useful regardless of whether the sentence has  mutation info
temp = pd.DataFrame(temp[:], columns=['WBPaper ID', 'Method', '* Genes', '* Gene-Variant combo', 'Mutation', 'Sentence'])
temp.to_csv("data/model_output/extracted_snippets_part3.csv", index=False, encoding='utf-8')